# The steps below will create and populate an FTS database, which we may want to do every once in a while just to be sure we're current.

Below will create a new fts5_new.db that is empty -- it is a standard fts database and contains content.

In [ ]:
import sqlite3
#db = sqlite3.connect('mylistmanager_s.db')
#db_c = db.cursor()
fts = sqlite3.connect('fts5.db')
fts_c = fts.cursor()
# uncomment below when you want to recreate the fts5 database
#fts.execute('CREATE VIRTUAL TABLE fts USING fts5 (title, note, lm_id UNINDEXED);')
# NOTE: don't need the below because deleted items just don't find matches in mylismanager_s.db
# I should thought periodically clean out deleted items
#fts.execute('CREATE VIRTUAL TABLE fts USING fts5 (title, note, lm_id UNINDEXED, deleted);') ##?

Below would create an fts database that would include tags

In [ ]:
import sqlite3
fts = sqlite3.connect('fts5.db')
fts_c = fts.cursor()
fts.execute('CREATE VIRTUAL TABLE fts USING fts5 (title, note, tag, lm_id UNINDEXED);')

Below just testing creating tag

In [ ]:
from lmdb_s import *
task = local_session.query(Task).get(4773)
keywords = task.keywords
tag = ",".join([k.name for k in keywords])
print(tag)

This is how you populate a new fts database that will be named whatever you named it above

In [ ]:
#fts_c = fts.cursor()

from lmdb_s import *
from time import sleep
tasks = local_session.query(Task)
for task in tasks:
    if task.deleted:
        continue
    title = task.title
    note = task.note
    if not note:
        note = ""
    title = title.replace("'", "''")
    note = note.replace("'", "''")
    try:
        # the title field is field 0 and the note field is field 1
        fts_c.execute(f"INSERT INTO fts (title, note, lm_id) VALUES (\'{title}\',\'{note}\', {task.id});")
    except Exception as e:
        print(f"INSERT INTO fts (title, note, lm_id) VALUES (\'{title}\',\'{note}\', {task.id});")
        print(e)
        
    sleep(.01) 
    

fts.commit()
print("done")

# This is how you populate a new fts database that includes tags

In [ ]:
import sqlite3
fts = sqlite3.connect('fts5.db')
#apparently add unique constraint so immediately below won't work
#fts.execute('CREATE VIRTUAL TABLE fts USING fts5 (title, note, tag, lm_id UNINDEXED UNIQUE);')
fts.execute('CREATE VIRTUAL TABLE fts USING fts5 (title, note, tag, lm_id UNINDEXED);')

In [ ]:
#fts_c = fts.cursor()

from lmdb_s import *
from time import sleep
tasks = local_session.query(Task)
for task in tasks:
    if task.deleted:
        continue
    title = task.title
    note = task.note
    if not note:
        note = ""
    title = title.replace("'", "''")
    note = note.replace("'", "''")
    #keywords = task.keywords
    tag = ",".join([k.name for k in task.keywords])
    try:
        # the title field is field 0 and the note field is field 1
        fts_c.execute(f"INSERT INTO fts (title, note, tag, lm_id) VALUES (\'{title}\', \'{note}\', \'{tag}\' ,{task.id});")
    except Exception as e:
        print(f"INSERT INTO fts (title, note, tag, lm_id) VALUES (\'{title}\', \'{note}\', \'{tag}\' ,{task.id});")
        print(e)
        
    sleep(.01) 
    

fts.commit()
print("done")

This demonstrates searching and retrieving ids using the FTS5 virtual table to retrieve matches

In [ ]:
fts_c.execute("SELECT lm_id FROM fts WHERE fts MATCH 'norm';")
rows = fts_c.fetchall()
print(rows)
#fts_c.close()
#fts.close()

This demonstrates highlighting the title (field is 0)

In [ ]:
fts = sqlite3.connect('fts5.db')
fts_c = fts.cursor()
fts_c.execute("SELECT lm_id, highlight(fts, 0, '[',']') FROM fts WHERE fts MATCH 'norm' AND lm_id=229;")
rows = fts_c.fetchall()
print(rows)

This demonstrates highlighting the note (field = 1)

In [ ]:
fts_c.execute("SELECT lm_id, highlight(fts, 1, '[',']') FROM fts WHERE fts MATCH 'norm' AND lm_id=5065;")
rows = fts_c.fetchall()
print(rows)